# conv2d

[from](https://blog.csdn.net/panghuzhenbang/article/details/129713598)

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn

def conv2d(inputs, in_channels, out_channels, kernel_size, stride = 1, padding = 0, weights = None, bias = None):
    if weights == None:
        weights = torch.randn([out_channels, in_channels, kernel_size[0], kernel_size[1]])
    if bias is None:
        bias = torch.zeros(out_channels)

    def fixed_padding(inputs, kernel_size):
        pad_total = kernel_size - 1
        pad_beg = pad_total / 2
        pad_end = pad_total - pad_beg
        padded_inputs = F.pad(inputs, [pad_beg, pad_end, pad_beg, pad_end], mode= 'constant', value = 0)
        return padded_inputs
        
    if padding:
        padded_inputs = F.pad(inputs, [padding, padding, padding, padding], mode='constant', value=0)
    else:
        padded_inputs = fixed_padding(inputs, kernel_size)

    n, c, w, h = padded_inputs.shape
    outputs = []

    for i, imgs in enumerate(padded_inputs):
        one_batch_out = []
        for j in range(out_channels):
            feature_map = []
            row = 0
            
            while row + kernel_size[0] <= h:
                row_feature_map = []
                col = 0

                while col + kernel_size[1] <= w:
                    point = [0 for ch in range(c)]
                    for ch in range(c):
                        for y in range(kernel_size[0]):
                            for x in range(kernel_size[1]):
                                point[ch] += imgs[ch][row+y][col+x] * weights[j][ch][y][x]
                    point = sum(point) + bias[j]
                    row_feature_map.append(point)
                    col += stride[1] if isinstance(stride, (list, tuple)) else stride
                feature_map.append(row_feature_map)
                row += stride[0] if isinstance(stride, (list, tuple)) else stride
            one_batch_out.append(feature_map)
        outputs.append(one_batch_out)
    
    return torch.tensor(outputs)


# pool2d

In [ ]:
import numpy as np

def pooling(inputs, pool_size, stride, mode = 'max'):
    c, w, h = inputs.shape
    k = pool_size
    outputs = np.zeros((c, (w - k) / stride + 1, (h - k) / stride + 1))

    if mode == 'max':
        for i in range(0, w - k + 1, stride):
            for j in range(0, h - k + 1, stride):
                outputs[:, i / stride, j / stride] = np.max(inputs[:, i:i+k, j:j+k], axis = (1,2))
        return outputs
    elif mode == 'avg':
        for i in range(0, w - k +1, stride):
            for j in range(0, h - k + 1, stride):
                outputs[:, i / stride, j / stride] = np.mean(inputs[:, i:i+k. j:j+k], axis = (1,2))
        return outputs
    else:
        raise ValueError('not support this mode, choose "max" or "avg" ')

def test():
    pool_size = 2
    stride = 2
    mode = 'max'
    inputs = np.arrange(1, 76).reshape((3, 5, 5))
    outputs = pooling(inputs, pool_size, stride, mode)
    print("outputs:{}".format(outputs.shape), '\n', outputs)    


# CNN

In [ ]:
import torch
from torch import nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 25, kernel_size=3),
            nn.BatchNorm2d(25),
            nn.ReLU(inplace=True)
        )

        self.layer2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(25, 50, kernel_size=3),
            nn.BatchNorm2d(50),
            nn.ReLU(inplace=True)
        )

        self.layer4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Sequential(
            nn.Linear(50 * 5 * 5, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


# RNN

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class ConvRNN(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size, dilation):
        super().__init__()
        pad_x = int(dilation * (kernel_size - 1) / 2)
        self.conv_x = nn.Conv2d(in_dim, out_dim, kernel_size, padding=pad_x, dilation=dilation)

        pad_h = int((kernel_size - 1) / 2)
        self.conv_h = nn.Conv2d(out_dim, out_dim, kernel_size, padding=pad_h)

        self.relu = nn.LeakyReLU(0.2)

    def forward(self, x, h = None):
        if h is None:
            h = F.tanh(self.conv_x(x))
        else:
            h = F.tanh(self.conv_x(x) + self.conv_h(h))
        
        h = self.relu(h)
        return h, h

# LSTM

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class ConvLSTM(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size, dilation):
        super().__init__()
        pad_x = int(dilation * (kernel_size - 1) / 2)
        self.conv_xf = nn.Conv2d(in_dim, out_dim, kernel_size, padding=pad_x, dilation=dilation)
        self.conv_xi = nn.Conv2d(in_dim, out_dim, kernel_size, padding=pad_x, dilation=dilation)
        self.conv_xo = nn.Conv2d(in_dim, out_dim, kernel_size, padding=pad_x, dilation=dilation)
        self.conv_xj = nn.Conv2d(in_dim, out_dim, kernel_size, padding=pad_x, dilation=dilation)

        pad_h = int((kernel_size - 1) / 2)
        self.conv_hf = nn.Conv2d(out_dim, out_dim, kernel_size=kernel_size, padding= pad_h)
        self.conv_hi = nn.Conv2d(out_dim, out_dim, kernel_size=kernel_size, padding= pad_h)
        self.conv_ho = nn.Conv2d(out_dim, out_dim, kernel_size=kernel_size, padding= pad_h)
        self.conv_hj = nn.Conv2d(out_dim, out_dim, kernel_size=kernel_size, padding= pad_h)

        self.relu = nn.LeakyReLU(0.2)
    
    def forward(self, x, pair = None):
        if pair is None:
            i = F.sigmoid(self.conv_xi(x))
            j = F.sigmoid(self.conv_xj(x))
            c = i * j
            o = F.sigmoid(self.conv_xo(x))
            h = o * c
        else:
            h, c = pair
            f = F.sigmoid(self.conv_xf(x) + self.conv_hf(h))
            i = F.sigmoid(self.conv_xi(x) + self.conv_hi(h))
            j = F.sigmoid(self.conv_xj(x) + self.conv_hj(h))
            c = f * c + i * j
            o = F.sigmoid(self.conv_xo(x) + self.conv_ho(h))
            h = o * F.tanh(c)
        
        h = self.relu(h)
        return h, [h, c]

# GRU

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvGRU(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size, dilation):
        super().__init__()
        pad_x  = int(dilation * (kernel_size - 1) / 2)
        self.conv_xz = nn.Conv2d(in_dim, out_dim, kernel_size, padding=pad_x, dilation=dilation)
        self.conv_xr = nn.Conv2d(in_dim, out_dim, kernel_size, padding=pad_x, dilation=dilation)
        self.conv_xn = nn.Conv2d(in_dim, out_dim, kernel_size, padding=pad_x, dilation=dilation)

        pad_h = int((kernel_size - 1) / 2)
        self.conv_hz = nn.Conv2d(out_dim, out_dim, kernel_size, padding=pad_h)
        self.conv_hr = nn.Conv2d(out_dim, out_dim, kernel_size, padding=pad_h)
        self.conv_hn = nn.Conv2d(out_dim, out_dim, kernel_size, padding=pad_h)

        self.relu = nn.LeakyReLU(0.2)

    def forward(self, x, h = None):
        if h is None:
            z = F.sigmoid(self.conv_xz(x))
            f = F.sigmoid(self.conv_xn(x))
            h = z * f
        else:
            z = F.sigmoid(self.conv_xnz(x) + self.conv_hz(h))
            r = F.sigmoid(self.conv_xr(x) + self.conv_hr(h))
            n = F.tanh(self.conv_xn(x) + self.conv_hn(r * h))
            h = (1 - z) * h + z * n

        h = self.relu(h)
        return h, h